In [89]:
# including error states
# phase flips errors E = |-+><-+| + |+-><+-|
# ancilla qubits (x, y) for distinguishing between the two types of errors

# NOTE is this form of an Isometry valid?
# idea is that it is simpler to leave out the error states until they are actually needed
# basically, the no error flag qubit is implicit
# U = |g+f, g+f><0| + |g-f, g-f><1| + |++x><+-| + |--y><-+|